### Dota 2 Winner Prediction: 
https://www.kaggle.com/c/mlcourse-dota2-win-predictionhttps://www.kaggle.com/c/mlcourse-dota2-win-prediction

(Download train_matches.jsonl and test_matches.jsonl from Kaggle)

In [1]:
import numpy as np
import pandas as pd
import pickle
import json
from matplotlib import pyplot as plt
%matplotlib inline

import lightgbm as lgb
from tqdm import tqdm_notebook
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, ShuffleSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [2]:
train_df = pd.read_csv('data/train_features.csv.zip', index_col='match_id_hash')
train_target = pd.read_csv('data/train_targets.csv.zip', index_col='match_id_hash')
test_df = pd.read_csv('data/test_features.csv.zip', index_col='match_id_hash')

X = train_df.copy()
X_test = test_df.copy()
y = train_target['radiant_win'].map({False: 0, True: 1})
shuffle = ShuffleSplit(n_splits=5, test_size=0.3, random_state=17)

def write_submission(y_test_pred, filename='untitled.csv'):
    df_submission = pd.DataFrame({'radiant_win_prob': y_test_pred}, index=test_df.index)
    df_submission.to_csv(filename)
    print('Submission saved to {}'.format(filename))   

In [3]:
def read_matches(filename):
    MATCHES_COUNT = {
        'data/test_matches.jsonl': 10000,
        'data/train_matches.jsonl': 39675,
    }
    total_matches = MATCHES_COUNT.get(filename)
    with open(filename) as fin:
        for line in tqdm_notebook(fin, total=total_matches):
            yield json.loads(line)
            
            
with open('data/heroes_dict.pickle', 'rb') as file:
    heroes_dict = pickle.load(file)

In [4]:
def add_new_features(X, matches_file):
    
    # Add summary of tower kills as feature
    
    for match in read_matches(matches_file):
        match_id_hash = match['match_id_hash']

        radiant_tower_kills = 0
        dire_tower_kills = 0
        for objective in match['objectives']:
            if objective['type'] == 'CHAT_MESSAGE_TOWER_KILL':
                if objective['team'] == 2:
                    radiant_tower_kills += 1
                if objective['team'] == 3:
                    dire_tower_kills += 1

        X.loc[match_id_hash, 'radiant_tower_kills'] = radiant_tower_kills
        X.loc[match_id_hash, 'dire_tower_kills'] = dire_tower_kills
        X.loc[match_id_hash, 'diff_tower_kills'] = radiant_tower_kills - dire_tower_kills

In [5]:
add_new_features(X, 'data/train_matches.jsonl')
add_new_features(X_test, 'data/test_matches.jsonl')

In [7]:
def show_importance(model,df,  head=10):
    return pd.DataFrame(model.feature_importances_, 
                 index=df.columns.values, 
                 columns=['importance']
    ).sort_values(by=['importance'], ascending=False)[:head]

In [8]:
def add_total_and_mean_features(df):
    interesting_features = ['kills', 'deaths', 'assists', 'gold', 'xp',
                            'lh', 'denies', 'level', 
                            'health', 'max_health', 'max_mana', 
                            'creeps_stacked', 'camps_stacked', 'rune_pickups', 'stuns',
                            'firstblood_claimed', 'teamfight_participation', 'towers_killed',
                            'roshans_killed', 'obs_placed', 'sen_placed']
    for feat in interesting_features:
        r_columns = [f'r{i}_{feat}' for i in range(1, 6)]
        d_columns = [f'd{i}_{feat}' for i in range(1, 6)]

        df['r_total_' + feat] = df[r_columns].sum(axis=1)
        df['d_total_' + feat] = df[d_columns].sum(axis=1)
        df['total_diff_' + feat] = df['r_total_' + feat] - df['d_total_' + feat]
        df['total_ratio_' + feat] = df['r_total_' + feat] / df['d_total_' + feat]
        
        df['r_std_' + feat] = df[r_columns].std(axis=1)
        df['d_std_' + feat] = df[d_columns].std(axis=1)
        df['std_diff_' + feat] = df['r_std_' + feat] - df['d_std_' + feat]
        df['std_ratio_' + feat] = df['r_std_' + feat] / df['d_std_' + feat]
        
        df['r_mean_' + feat] = df[r_columns].mean(axis=1)
        df['d_mean_' + feat] = df[d_columns].mean(axis=1)
        df['mean_diff_' + feat] = df['r_mean_' + feat] - df['d_mean_' + feat]
        df['mean_ratio_' + feat] = df['r_mean_' + feat] / df['d_mean_' + feat]
        
        df['r_max_' + feat] = df[r_columns].max(axis=1)
        df['d_max_' + feat] = df[d_columns].max(axis=1)
        df['max_diff_' + feat] = df['r_max_' + feat] - df['d_max_' + feat]
        df['max_ratio_' + feat] = df['r_max_' + feat] / df['d_max_' + feat]
         

    r_columns = [f'r{i}_hero_id' for i in range(1, 6)]
    d_columns = [f'd{i}_hero_id' for i in range(1, 6)]
    for col in tqdm_notebook(r_columns):
        for match_hash_id in df.index:
            id = df.loc[match_hash_id, col]
            df.loc[match_hash_id, f'r_hero_id_{id}'] = 1
    for col in tqdm_notebook(d_columns):
        for match_hash_id in df.index:
            id = df.loc[match_hash_id, col]
            df.loc[match_hash_id, f'd_hero_id_{id}'] = 1
    df.fillna(0)

In [9]:
new_X = X.copy()
add_total_and_mean_features(new_X)
new_X.shape

(39675, 814)

In [10]:
new_X = new_X.loc[:, 'radiant_tower_kills':]
new_X = pd.concat([X[['game_time', 'game_mode']], new_X], axis=1)
new_X = new_X.replace([np.nan, np.inf, -np.inf], 0)

In [12]:
new_X.columns.values

array(['game_time', 'game_mode', 'radiant_tower_kills',
       'dire_tower_kills', 'diff_tower_kills', 'r_total_kills',
       'd_total_kills', 'total_diff_kills', 'total_ratio_kills',
       'r_std_kills', 'd_std_kills', 'std_diff_kills', 'std_ratio_kills',
       'r_mean_kills', 'd_mean_kills', 'mean_diff_kills',
       'mean_ratio_kills', 'r_max_kills', 'd_max_kills', 'max_diff_kills',
       'max_ratio_kills', 'r_total_deaths', 'd_total_deaths',
       'total_diff_deaths', 'total_ratio_deaths', 'r_std_deaths',
       'd_std_deaths', 'std_diff_deaths', 'std_ratio_deaths',
       'r_mean_deaths', 'd_mean_deaths', 'mean_diff_deaths',
       'mean_ratio_deaths', 'r_max_deaths', 'd_max_deaths',
       'max_diff_deaths', 'max_ratio_deaths', 'r_total_assists',
       'd_total_assists', 'total_diff_assists', 'total_ratio_assists',
       'r_std_assists', 'd_std_assists', 'std_diff_assists',
       'std_ratio_assists', 'r_mean_assists', 'd_mean_assists',
       'mean_diff_assists', 'mean_ra

In [24]:
def add_new_json_features(df, file):
    
    for match in read_matches(file):
        match_id_hash = match['match_id_hash']
        
        dist_sum_r = 0
        dist_sum_d = 0
        r_pred_vict = 0
        d_pred_vict = 0
        r_randomed = 0
        d_randomed = 0
        r_killstreaks = 0
        d_killstreaks = 0
        for player in match['players']:
            if (player['player_slot'] < 128):
                if (player['pred_vict'] == True):
                    r_pred_vict += 1
                if (player['randomed'] == True):
                    r_randomed += 1
                for streak in player['kill_streaks']:
                    r_killstreaks += player['kill_streaks'][streak]
                for ward in player['obs_log']:
                    dist_sum_r += (ward['x'] - 66) ** 2 + (ward['y'] - 66) ** 2
            else:
                if (player['pred_vict'] == True):
                    d_pred_vict += 1
                if (player['randomed'] == True):
                    d_randomed += 1
                for streak in player['kill_streaks']:
                    d_killstreaks += player['kill_streaks'][streak]
                for ward in player['obs_log']:
                    dist_sum_d += (ward['x'] - 186) ** 2 + (ward['y'] - 186) ** 2 
        
        df.loc[match_id_hash, 'r_ward_dist'] =  dist_sum_r
        df.loc[match_id_hash, 'd_ward_dist'] =  dist_sum_d
        df.loc[match_id_hash, 'r_pred_vict'] =  r_pred_vict
        df.loc[match_id_hash, 'd_pred_vict'] =  d_pred_vict
        df.loc[match_id_hash, 'r_randomed'] =  r_randomed
        df.loc[match_id_hash, 'd_randomed'] =  d_randomed
        df.loc[match_id_hash, 'r_streaks'] = r_killstreaks
        df.loc[match_id_hash, 'd_streaks'] = d_killstreaks
        
        

In [25]:
v2_X = new_X.copy()
add_new_json_features(v2_X, 'data/train_matches.jsonl')

In [33]:
v2_X['streaks_diff'] = v2_X['r_streaks'] - v2_X['d_streaks']
v2_X.head()

,game_time,game_mode,radiant_tower_kills,dire_tower_kills,diff_tower_kills,r_total_kills,d_total_kills,total_diff_kills,total_ratio_kills,r_std_kills,...,d_hero_id_55,r_ward_dist,d_ward_dist,r_pred_vict,d_pred_vict,r_randomed,d_randomed,r_streaks,d_streaks,streaks_diff
match_id_hash,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,0.0,0.0,0.0,0,1,-1,0.000000,0.000000,...,0.0,0.0,12392.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0
b9c57c450ce74a2af79c9ce96fac144d,658,4,2.0,0.0,2.0,16,3,13,5.333333,2.489980,...,0.0,35340.0,9016.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0
6db558535151ea18ca70a6892197db41,21,23,0.0,0.0,0.0,0,0,0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46a0ddce8f7ed2a8d9bd5edcbb925682,576,22,0.0,0.0,0.0,6,1,5,6.000000,1.095445,...,0.0,37076.0,31088.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
b1b35ff97723d9b7ade1c9c3cf48f770,453,22,0.0,0.0,0.0,2,4,-2,0.500000,0.547723,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


#### Fitting CatBoost

In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(v2_X, y, test_size=0.3, random_state=17)

In [35]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(n_estimators=1000)
cat_features = list(range(341, 571)) #hero ids
model.fit(X_train, y_train, cat_features)
cat_pred = model.predict_proba(X_valid)[:, 1]

Learning rate set to 0.042921
0:	learn: 0.6770792	total: 1.41s	remaining: 23m 30s
1:	learn: 0.6629761	total: 2.29s	remaining: 19m
2:	learn: 0.6501314	total: 3.59s	remaining: 19m 54s
3:	learn: 0.6383859	total: 4.42s	remaining: 18m 20s
4:	learn: 0.6273101	total: 5.38s	remaining: 17m 49s
5:	learn: 0.6185291	total: 6.1s	remaining: 16m 50s
6:	learn: 0.6104135	total: 6.81s	remaining: 16m 6s
7:	learn: 0.6018315	total: 7.81s	remaining: 16m 8s
8:	learn: 0.5942575	total: 8.52s	remaining: 15m 38s
9:	learn: 0.5874385	total: 9.26s	remaining: 15m 16s
10:	learn: 0.5814828	total: 9.95s	remaining: 14m 54s
11:	learn: 0.5764544	total: 10.7s	remaining: 14m 36s
12:	learn: 0.5720115	total: 11.4s	remaining: 14m 23s
13:	learn: 0.5665920	total: 12.5s	remaining: 14m 38s
14:	learn: 0.5620428	total: 13.5s	remaining: 14m 46s
15:	learn: 0.5579493	total: 14.5s	remaining: 14m 49s
16:	learn: 0.5538542	total: 15.3s	remaining: 14m 45s
17:	learn: 0.5506096	total: 16.1s	remaining: 14m 38s
18:	learn: 0.5479637	total: 16.9s

154:	learn: 0.4835867	total: 1m 47s	remaining: 9m 47s
155:	learn: 0.4833970	total: 1m 48s	remaining: 9m 46s
156:	learn: 0.4832857	total: 1m 49s	remaining: 9m 45s
157:	learn: 0.4831053	total: 1m 49s	remaining: 9m 44s
158:	learn: 0.4829313	total: 1m 50s	remaining: 9m 43s
159:	learn: 0.4828409	total: 1m 50s	remaining: 9m 42s
160:	learn: 0.4826977	total: 1m 51s	remaining: 9m 41s
161:	learn: 0.4825495	total: 1m 52s	remaining: 9m 40s
162:	learn: 0.4824499	total: 1m 52s	remaining: 9m 39s
163:	learn: 0.4823086	total: 1m 53s	remaining: 9m 39s
164:	learn: 0.4822148	total: 1m 54s	remaining: 9m 39s
165:	learn: 0.4821677	total: 1m 55s	remaining: 9m 38s
166:	learn: 0.4820360	total: 1m 55s	remaining: 9m 38s
167:	learn: 0.4819322	total: 1m 56s	remaining: 9m 37s
168:	learn: 0.4818737	total: 1m 57s	remaining: 9m 37s
169:	learn: 0.4817198	total: 1m 58s	remaining: 9m 36s
170:	learn: 0.4815888	total: 1m 58s	remaining: 9m 35s
171:	learn: 0.4814772	total: 1m 59s	remaining: 9m 34s
172:	learn: 0.4813429	total:

308:	learn: 0.4674430	total: 3m 34s	remaining: 8m
309:	learn: 0.4673191	total: 3m 35s	remaining: 8m
310:	learn: 0.4671940	total: 3m 36s	remaining: 7m 59s
311:	learn: 0.4671168	total: 3m 37s	remaining: 7m 58s
312:	learn: 0.4669952	total: 3m 37s	remaining: 7m 58s
313:	learn: 0.4669125	total: 3m 38s	remaining: 7m 57s
314:	learn: 0.4667576	total: 3m 39s	remaining: 7m 56s
315:	learn: 0.4666587	total: 3m 39s	remaining: 7m 55s
316:	learn: 0.4665941	total: 3m 40s	remaining: 7m 55s
317:	learn: 0.4664920	total: 3m 41s	remaining: 7m 54s
318:	learn: 0.4663682	total: 3m 41s	remaining: 7m 53s
319:	learn: 0.4661720	total: 3m 42s	remaining: 7m 53s
320:	learn: 0.4661268	total: 3m 43s	remaining: 7m 52s
321:	learn: 0.4660398	total: 3m 43s	remaining: 7m 51s
322:	learn: 0.4659503	total: 3m 44s	remaining: 7m 50s
323:	learn: 0.4658056	total: 3m 45s	remaining: 7m 49s
324:	learn: 0.4657510	total: 3m 45s	remaining: 7m 48s
325:	learn: 0.4656633	total: 3m 46s	remaining: 7m 48s
326:	learn: 0.4655187	total: 3m 47s	

461:	learn: 0.4525589	total: 5m 22s	remaining: 6m 15s
462:	learn: 0.4525298	total: 5m 22s	remaining: 6m 14s
463:	learn: 0.4524372	total: 5m 23s	remaining: 6m 13s
464:	learn: 0.4523266	total: 5m 23s	remaining: 6m 12s
465:	learn: 0.4522788	total: 5m 24s	remaining: 6m 11s
466:	learn: 0.4521634	total: 5m 25s	remaining: 6m 11s
467:	learn: 0.4520955	total: 5m 25s	remaining: 6m 10s
468:	learn: 0.4520082	total: 5m 26s	remaining: 6m 9s
469:	learn: 0.4518947	total: 5m 27s	remaining: 6m 8s
470:	learn: 0.4518141	total: 5m 27s	remaining: 6m 8s
471:	learn: 0.4517440	total: 5m 28s	remaining: 6m 7s
472:	learn: 0.4516743	total: 5m 29s	remaining: 6m 6s
473:	learn: 0.4516576	total: 5m 29s	remaining: 6m 6s
474:	learn: 0.4515408	total: 5m 30s	remaining: 6m 5s
475:	learn: 0.4514448	total: 5m 31s	remaining: 6m 5s
476:	learn: 0.4513722	total: 5m 32s	remaining: 6m 4s
477:	learn: 0.4513012	total: 5m 33s	remaining: 6m 3s
478:	learn: 0.4512433	total: 5m 33s	remaining: 6m 2s
479:	learn: 0.4511866	total: 5m 34s	rem

615:	learn: 0.4404287	total: 7m 12s	remaining: 4m 29s
616:	learn: 0.4403680	total: 7m 13s	remaining: 4m 28s
617:	learn: 0.4403089	total: 7m 13s	remaining: 4m 28s
618:	learn: 0.4402960	total: 7m 14s	remaining: 4m 27s
619:	learn: 0.4402096	total: 7m 15s	remaining: 4m 26s
620:	learn: 0.4400857	total: 7m 16s	remaining: 4m 26s
621:	learn: 0.4400186	total: 7m 16s	remaining: 4m 25s
622:	learn: 0.4399301	total: 7m 17s	remaining: 4m 24s
623:	learn: 0.4399002	total: 7m 18s	remaining: 4m 24s
624:	learn: 0.4398781	total: 7m 18s	remaining: 4m 23s
625:	learn: 0.4398224	total: 7m 19s	remaining: 4m 22s
626:	learn: 0.4397041	total: 7m 20s	remaining: 4m 22s
627:	learn: 0.4395969	total: 7m 21s	remaining: 4m 21s
628:	learn: 0.4395106	total: 7m 22s	remaining: 4m 20s
629:	learn: 0.4394836	total: 7m 22s	remaining: 4m 20s
630:	learn: 0.4394165	total: 7m 23s	remaining: 4m 19s
631:	learn: 0.4393210	total: 7m 24s	remaining: 4m 18s
632:	learn: 0.4392817	total: 7m 24s	remaining: 4m 17s
633:	learn: 0.4392139	total:

768:	learn: 0.4297904	total: 8m 59s	remaining: 2m 42s
769:	learn: 0.4297787	total: 8m 59s	remaining: 2m 41s
770:	learn: 0.4297157	total: 9m	remaining: 2m 40s
771:	learn: 0.4295823	total: 9m 1s	remaining: 2m 39s
772:	learn: 0.4295513	total: 9m 1s	remaining: 2m 39s
773:	learn: 0.4295240	total: 9m 2s	remaining: 2m 38s
774:	learn: 0.4294973	total: 9m 3s	remaining: 2m 37s
775:	learn: 0.4294591	total: 9m 3s	remaining: 2m 36s
776:	learn: 0.4294484	total: 9m 4s	remaining: 2m 36s
777:	learn: 0.4293812	total: 9m 4s	remaining: 2m 35s
778:	learn: 0.4292962	total: 9m 5s	remaining: 2m 34s
779:	learn: 0.4292452	total: 9m 6s	remaining: 2m 34s
780:	learn: 0.4291560	total: 9m 7s	remaining: 2m 33s
781:	learn: 0.4291123	total: 9m 7s	remaining: 2m 32s
782:	learn: 0.4290341	total: 9m 8s	remaining: 2m 31s
783:	learn: 0.4290286	total: 9m 8s	remaining: 2m 31s
784:	learn: 0.4289669	total: 9m 9s	remaining: 2m 30s
785:	learn: 0.4288917	total: 9m 10s	remaining: 2m 29s
786:	learn: 0.4287949	total: 9m 10s	remaining:

921:	learn: 0.4207424	total: 10m 40s	remaining: 54.2s
922:	learn: 0.4206645	total: 10m 41s	remaining: 53.5s
923:	learn: 0.4206409	total: 10m 42s	remaining: 52.8s
924:	learn: 0.4206357	total: 10m 42s	remaining: 52.1s
925:	learn: 0.4205633	total: 10m 43s	remaining: 51.4s
926:	learn: 0.4205153	total: 10m 43s	remaining: 50.7s
927:	learn: 0.4205054	total: 10m 44s	remaining: 50s
928:	learn: 0.4204133	total: 10m 45s	remaining: 49.3s
929:	learn: 0.4203433	total: 10m 45s	remaining: 48.6s
930:	learn: 0.4202981	total: 10m 46s	remaining: 47.9s
931:	learn: 0.4202947	total: 10m 46s	remaining: 47.2s
932:	learn: 0.4202431	total: 10m 47s	remaining: 46.5s
933:	learn: 0.4202070	total: 10m 48s	remaining: 45.8s
934:	learn: 0.4202018	total: 10m 48s	remaining: 45.1s
935:	learn: 0.4201908	total: 10m 49s	remaining: 44.4s
936:	learn: 0.4201658	total: 10m 50s	remaining: 43.7s
937:	learn: 0.4200955	total: 10m 50s	remaining: 43s
938:	learn: 0.4200108	total: 10m 51s	remaining: 42.3s
939:	learn: 0.4198799	total: 10m

In [36]:
roc_auc_score(y_valid, cat_pred)

0.8386459586240927

#### Transforming X_test and fit for submit

In [38]:
new_X_test = X_test.copy()
add_total_and_mean_features(new_X_test)
new_X_test = new_X_test.loc[:, 'radiant_tower_kills':]
new_X_test = pd.concat([X_test['game_time'], new_X_test], axis=1)
new_X_test = new_X_test.replace([np.nan, np.inf, -np.inf], 0)

In [37]:
show_importance(model, v2_X, 20)

,importance
total_diff_gold,9.549166
total_ratio_gold,9.077669
diff_tower_kills,3.771062
total_ratio_xp,3.400121
total_diff_xp,2.789324
total_ratio_level,2.296881
total_diff_level,2.029014
radiant_tower_kills,1.973046
dire_tower_kills,1.758270
d_hero_id_32,1.569209


In [113]:
heroes_dict[32]

'npc_dota_hero_riki'

In [39]:
v2_X_test = new_X_test.copy()
add_new_json_features(v2_X_test, 'data/test_matches.jsonl')

In [40]:
model.fit(v2_X, y, cat_features)
cat_test_pred = model.predict_proba(v2_X_test)[:, 1]
write_submission(cat_test_pred, 'cat_3_dummies_wards.csv')

Learning rate set to 0.047479
0:	learn: 0.6750696	total: 730ms	remaining: 12m 9s
1:	learn: 0.6607863	total: 1.35s	remaining: 11m 15s
2:	learn: 0.6483546	total: 2.05s	remaining: 11m 21s
3:	learn: 0.6365078	total: 2.58s	remaining: 10m 43s
4:	learn: 0.6242231	total: 3.29s	remaining: 10m 54s
5:	learn: 0.6146292	total: 3.78s	remaining: 10m 25s
6:	learn: 0.6049998	total: 4.24s	remaining: 10m
7:	learn: 0.5967503	total: 4.86s	remaining: 10m 2s
8:	learn: 0.5896238	total: 5.37s	remaining: 9m 50s
9:	learn: 0.5835017	total: 5.88s	remaining: 9m 42s
10:	learn: 0.5769890	total: 6.5s	remaining: 9m 44s
11:	learn: 0.5710023	total: 7.11s	remaining: 9m 45s
12:	learn: 0.5665058	total: 7.68s	remaining: 9m 42s
13:	learn: 0.5627335	total: 8.36s	remaining: 9m 48s
14:	learn: 0.5588885	total: 8.88s	remaining: 9m 43s
15:	learn: 0.5551987	total: 9.4s	remaining: 9m 38s
16:	learn: 0.5510881	total: 9.97s	remaining: 9m 36s
17:	learn: 0.5472997	total: 10.6s	remaining: 9m 37s
18:	learn: 0.5446418	total: 11.2s	remaining:

157:	learn: 0.4831266	total: 1m 23s	remaining: 7m 27s
158:	learn: 0.4829630	total: 1m 24s	remaining: 7m 26s
159:	learn: 0.4828400	total: 1m 24s	remaining: 7m 26s
160:	learn: 0.4827438	total: 1m 25s	remaining: 7m 25s
161:	learn: 0.4826560	total: 1m 26s	remaining: 7m 24s
162:	learn: 0.4825287	total: 1m 26s	remaining: 7m 24s
163:	learn: 0.4824139	total: 1m 26s	remaining: 7m 23s
164:	learn: 0.4823511	total: 1m 27s	remaining: 7m 22s
165:	learn: 0.4822311	total: 1m 27s	remaining: 7m 21s
166:	learn: 0.4821351	total: 1m 28s	remaining: 7m 20s
167:	learn: 0.4820395	total: 1m 28s	remaining: 7m 19s
168:	learn: 0.4819319	total: 1m 29s	remaining: 7m 19s
169:	learn: 0.4818311	total: 1m 29s	remaining: 7m 18s
170:	learn: 0.4817385	total: 1m 30s	remaining: 7m 17s
171:	learn: 0.4816757	total: 1m 30s	remaining: 7m 16s
172:	learn: 0.4815694	total: 1m 31s	remaining: 7m 16s
173:	learn: 0.4814558	total: 1m 31s	remaining: 7m 15s
174:	learn: 0.4813772	total: 1m 32s	remaining: 7m 14s
175:	learn: 0.4812742	total:

310:	learn: 0.4684856	total: 2m 42s	remaining: 5m 58s
311:	learn: 0.4684290	total: 2m 42s	remaining: 5m 58s
312:	learn: 0.4683633	total: 2m 42s	remaining: 5m 57s
313:	learn: 0.4682820	total: 2m 43s	remaining: 5m 57s
314:	learn: 0.4681754	total: 2m 43s	remaining: 5m 56s
315:	learn: 0.4680724	total: 2m 44s	remaining: 5m 56s
316:	learn: 0.4679676	total: 2m 45s	remaining: 5m 55s
317:	learn: 0.4678871	total: 2m 45s	remaining: 5m 55s
318:	learn: 0.4677599	total: 2m 46s	remaining: 5m 55s
319:	learn: 0.4676783	total: 2m 47s	remaining: 5m 54s
320:	learn: 0.4676037	total: 2m 47s	remaining: 5m 54s
321:	learn: 0.4675237	total: 2m 48s	remaining: 5m 54s
322:	learn: 0.4674480	total: 2m 48s	remaining: 5m 53s
323:	learn: 0.4673687	total: 2m 49s	remaining: 5m 53s
324:	learn: 0.4673072	total: 2m 49s	remaining: 5m 52s
325:	learn: 0.4672575	total: 2m 50s	remaining: 5m 51s
326:	learn: 0.4671644	total: 2m 50s	remaining: 5m 51s
327:	learn: 0.4670486	total: 2m 51s	remaining: 5m 50s
328:	learn: 0.4669939	total:

463:	learn: 0.4571272	total: 3m 53s	remaining: 4m 29s
464:	learn: 0.4570610	total: 3m 54s	remaining: 4m 29s
465:	learn: 0.4570230	total: 3m 54s	remaining: 4m 28s
466:	learn: 0.4569522	total: 3m 54s	remaining: 4m 28s
467:	learn: 0.4568966	total: 3m 55s	remaining: 4m 27s
468:	learn: 0.4568367	total: 3m 55s	remaining: 4m 27s
469:	learn: 0.4567819	total: 3m 56s	remaining: 4m 26s
470:	learn: 0.4567132	total: 3m 56s	remaining: 4m 25s
471:	learn: 0.4566750	total: 3m 57s	remaining: 4m 25s
472:	learn: 0.4566202	total: 3m 57s	remaining: 4m 24s
473:	learn: 0.4565317	total: 3m 58s	remaining: 4m 24s
474:	learn: 0.4564290	total: 3m 58s	remaining: 4m 23s
475:	learn: 0.4563643	total: 3m 59s	remaining: 4m 23s
476:	learn: 0.4563333	total: 3m 59s	remaining: 4m 22s
477:	learn: 0.4562763	total: 3m 59s	remaining: 4m 22s
478:	learn: 0.4562070	total: 4m	remaining: 4m 21s
479:	learn: 0.4561331	total: 4m	remaining: 4m 20s
480:	learn: 0.4561035	total: 4m 1s	remaining: 4m 20s
481:	learn: 0.4560276	total: 4m 1s	re

617:	learn: 0.4475222	total: 5m 11s	remaining: 3m 12s
618:	learn: 0.4474343	total: 5m 12s	remaining: 3m 12s
619:	learn: 0.4473779	total: 5m 12s	remaining: 3m 11s
620:	learn: 0.4473218	total: 5m 13s	remaining: 3m 11s
621:	learn: 0.4472635	total: 5m 13s	remaining: 3m 10s
622:	learn: 0.4472085	total: 5m 14s	remaining: 3m 10s
623:	learn: 0.4470644	total: 5m 15s	remaining: 3m 9s
624:	learn: 0.4469734	total: 5m 15s	remaining: 3m 9s
625:	learn: 0.4469193	total: 5m 16s	remaining: 3m 8s
626:	learn: 0.4468141	total: 5m 16s	remaining: 3m 8s
627:	learn: 0.4467303	total: 5m 17s	remaining: 3m 8s
628:	learn: 0.4467215	total: 5m 18s	remaining: 3m 7s
629:	learn: 0.4466380	total: 5m 19s	remaining: 3m 7s
630:	learn: 0.4465416	total: 5m 19s	remaining: 3m 6s
631:	learn: 0.4464880	total: 5m 20s	remaining: 3m 6s
632:	learn: 0.4464611	total: 5m 21s	remaining: 3m 6s
633:	learn: 0.4464407	total: 5m 22s	remaining: 3m 5s
634:	learn: 0.4464038	total: 5m 22s	remaining: 3m 5s
635:	learn: 0.4463753	total: 5m 23s	rema

771:	learn: 0.4398293	total: 6m 41s	remaining: 1m 58s
772:	learn: 0.4397700	total: 6m 41s	remaining: 1m 58s
773:	learn: 0.4397164	total: 6m 42s	remaining: 1m 57s
774:	learn: 0.4396576	total: 6m 43s	remaining: 1m 57s
775:	learn: 0.4396456	total: 6m 43s	remaining: 1m 56s
776:	learn: 0.4395937	total: 6m 44s	remaining: 1m 55s
777:	learn: 0.4395196	total: 6m 44s	remaining: 1m 55s
778:	learn: 0.4394927	total: 6m 45s	remaining: 1m 54s
779:	learn: 0.4394365	total: 6m 45s	remaining: 1m 54s
780:	learn: 0.4393838	total: 6m 46s	remaining: 1m 53s
781:	learn: 0.4393707	total: 6m 46s	remaining: 1m 53s
782:	learn: 0.4393477	total: 6m 47s	remaining: 1m 52s
783:	learn: 0.4392737	total: 6m 47s	remaining: 1m 52s
784:	learn: 0.4392391	total: 6m 48s	remaining: 1m 51s
785:	learn: 0.4391297	total: 6m 49s	remaining: 1m 51s
786:	learn: 0.4390677	total: 6m 49s	remaining: 1m 50s
787:	learn: 0.4389747	total: 6m 50s	remaining: 1m 50s
788:	learn: 0.4389219	total: 6m 50s	remaining: 1m 49s
789:	learn: 0.4388392	total:

925:	learn: 0.4325360	total: 7m 51s	remaining: 37.7s
926:	learn: 0.4325291	total: 7m 52s	remaining: 37.2s
927:	learn: 0.4324882	total: 7m 52s	remaining: 36.7s
928:	learn: 0.4324784	total: 7m 52s	remaining: 36.1s
929:	learn: 0.4324700	total: 7m 53s	remaining: 35.6s
930:	learn: 0.4324650	total: 7m 53s	remaining: 35.1s
931:	learn: 0.4324343	total: 7m 54s	remaining: 34.6s
932:	learn: 0.4324220	total: 7m 54s	remaining: 34.1s
933:	learn: 0.4324199	total: 7m 54s	remaining: 33.6s
934:	learn: 0.4323354	total: 7m 55s	remaining: 33s
935:	learn: 0.4322877	total: 7m 55s	remaining: 32.5s
936:	learn: 0.4322832	total: 7m 56s	remaining: 32s
937:	learn: 0.4321865	total: 7m 56s	remaining: 31.5s
938:	learn: 0.4321429	total: 7m 57s	remaining: 31s
939:	learn: 0.4320687	total: 7m 57s	remaining: 30.5s
940:	learn: 0.4320031	total: 7m 58s	remaining: 30s
941:	learn: 0.4319686	total: 7m 58s	remaining: 29.5s
942:	learn: 0.4319679	total: 7m 58s	remaining: 28.9s
943:	learn: 0.4319118	total: 7m 59s	remaining: 28.4s
9